Problem 1

In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_excel("lab3_excel.xlsx")

In [32]:
df['x (m)']

0     0.0
1     0.4
2     0.8
3     1.2
4     1.6
5     2.0
6     2.4
7     2.8
8     3.2
9     3.6
10    4.0
11    4.4
12    4.8
Name: x (m), dtype: float64

In [26]:
new_df

,Xm,Am^2
0,0.210,0.04410
1,0.430,0.17200
2,0.550,0.36300
3,0.635,0.44450
4,0.615,0.40590
5,0.590,0.38350
6,0.725,0.44950
7,0.865,0.47575
8,0.770,0.40810
9,0.665,0.29925


In [36]:
new_df

,Xm,A
0,0.210,0.04410
1,0.430,0.17200
2,0.550,0.36300
3,0.635,0.44450
4,0.615,0.40590
5,0.590,0.38350
6,0.725,0.44950
7,0.865,0.47575
8,0.770,0.40810
9,0.665,0.29925


In [33]:
new_df = pd.DataFrame()

new_df['Xm'] = (np.array(df[column].iloc[1:])+np.array(df[column].iloc[:-1]))/2
new_df['A'] = np.array(new_df['Xm'])*np.array(df['d (m)'].iloc[1:])


In [22]:
new_df

,Xm,Am^2
0,0.210,0.04410
1,0.430,0.17200
2,0.550,0.36300
3,0.635,0.44450
4,0.615,0.40590
5,0.590,0.38350
6,0.725,0.44950
7,0.865,0.47575
8,0.770,0.40810
9,0.665,0.29925


In [7]:
for i in df.items():
    print(i[1])

0     0.0
1     0.4
2     0.8
3     1.2
4     1.6
5     2.0
6     2.4
7     2.8
8     3.2
9     3.6
10    4.0
11    4.4
12    4.8
Name: x (m), dtype: float64
0     0.00
1     0.21
2     0.40
3     0.66
4     0.70
5     0.66
6     0.65
7     0.62
8     0.55
9     0.53
10    0.45
11    0.24
12    0.00
Name: d (m), dtype: float64
0     0.6
1     0.6
2     0.6
3     0.6
4     0.6
5     0.6
6     0.6
7     0.6
8     0.6
9     0.6
10    0.6
11    0.6
12    0.6
Name: Stage (ft), dtype: float64
0     0.00
1     0.42
2     0.44
3     0.66
4     0.61
5     0.62
6     0.56
7     0.89
8     0.84
9     0.70
10    0.63
11    0.53
12    0.00
Name: v (m/s), dtype: float64
